<a href="https://colab.research.google.com/github/namwootree/Portfolio/blob/main/Alphaco_(Deep_Learning_Boot_Camp)/Long-Term%20Program/Text_Summarization/Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting

In [1]:
!pip install transformers
!pip install sentencepiece
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

import transformers
from transformers import pipeline
import random

import nltk
from nltk.tokenize import sent_tokenize

from tqdm.notebook import tqdm

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
train = pd.read_csv('/content/drive/MyDrive/장기 프로젝트/문서 요약/train_df.csv')

In [6]:
train['context'] = train['context'].str.replace('.', '. ').replace('?', '? ').replace('없습니다.', '없습니다')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [7]:
train.head()

,Unnamed: 0,id,title,region,agenda,context,summary,total
0,1000,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_1,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 ...,제207회 완주군의회 임시회 제1차 본회의 개의 선포.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 의석을 정돈하여 주시기 ...
1,1001,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_2,의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 ...,제207회 완주군의회 임시회 회기는 8월 26일부터 9월 4일까지 10일간으로 가결됨.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 의사팀장 수고하셨습니다....
2,1002,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_3,다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...,제207회 완주군의회 임시회 회의록 서명의원으로 최등원 의원과 박웅배 의원이 선출됨.,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 다음은 의사일정 제2항 ...
3,1003,2000,제207회 완주군의회(임시회) 제 1 차 본회의회의록,완주,AGENDA_4,다음은 의사일정 제3항 본회의 휴회의 건을 상정합니다. 상임의원회 의정활동을 위하...,8월 27일부터 9월 3일까지 8일간 휴회가 가결됨. 제2차 본회의는 9월 4일 오...,제207회 완주군의회(임시회) 제 1 차 본회의회의록 완주 다음은 의사일정 제3항 ...
4,1004,2001,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록,완주,AGENDA_1,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제251회 완주군의회 제1차 ...,제251회 완주군의회 제1차 정례회 제1차 본회의 개의 선포.,제251회 완주군의회(제1차 정례회) 제1차 본 회 의 회 의 록 완주 의석을 정돈...


# Random Insertion

In [ ]:
dict_random_insertion = {
    'id':list(train['id']),
    'title':list(train['title']),
    'region':list(train['agenda']),
    'context':[],
    'summary':list(train['summary'])
}


unmasker = pipeline('fill-mask', model='klue/roberta-base')

for input_text in tqdm(list(train['context'])):

  new_text = ''
  
  sentence_list = sent_tokenize(input_text)

  for sentence in sentence_list:

    orig_text_list = sentence.split()
    len_input = len(orig_text_list)
    if len_input <= 2:

      new_text_list = orig_text_list[:1] + ['[MASK]'] + orig_text_list[1:]
      new_mask_sent = ' '.join(new_text_list)

      try:
        augmented_text_list = unmasker(new_mask_sent)
        augmented_text = augmented_text_list[0]['sequence']

        new_augmented_text = ' ' + augmented_text
        new_text += new_augmented_text
        new_text = new_text.lstrip(' ')
      except:
        print(f'오류 : {new_mask_sent}')
        new_text += ' ' + sentence
        pass

    else:
      rand_idx = random.randint(1, len_input-2)

      new_text_list = orig_text_list[:rand_idx] + ['[MASK]'] + orig_text_list[rand_idx:]
      new_mask_sent = ' '.join(new_text_list)

      try:
        augmented_text_list = unmasker(new_mask_sent)
        augmented_text = augmented_text_list[0]['sequence']

        new_augmented_text = ' ' + augmented_text
        new_text += new_augmented_text
        new_text = new_text.lstrip(' ')
      except:
        print(f'오류 : {new_mask_sent}')
        new_text += ' ' + sentence
        pass

  dict_random_insertion['context'].append(new_text+'.')

df_random_insertion = pd.DataFrame(dict_random_insertion)
df_random_insertion.to_csv('df_random_insertion.csv')
df_random_insertion.head()

  0%|          | 0/2994 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (677 > 512). Running this sequence through the model will result in indexing errors


오류 : 다음은 의원 발의 의안으로 이재길 의원님이 대표 발의하신 청주시장 및 관계공무원 출석요구의 건, 안흥수 의원님이 대표 발의하신 ｢청주시 의사상자 등 예우 및 지원에 관한 조례안｣, 변창수 의원님이 대표 발의하신 ｢청주시 장애인공무원 편의지원 조례안｣, 임기중 의원님이 대표 발의하신 ｢청주시 출산장려 및 양육에 대한 지원 조례 일부개정조례안｣, 서지한 의원님이 대표 발의하신 ｢청주시립요양병원 설치 및 운영에 관한 조례 일부개정조례안｣, 육미선 의원님이 대표 발의하신 ｢청주시 마을방송시스템 설치 지원에 관한 조례안｣, 최진현 의원님이 대표 발의하신 ｢청주시 주민자치센터 설치 및 운영 조례 일부개정조례안｣, 맹순자 의원님이 대표 발의하신 ｢청주시 지방재정계획심의위원회 설치ㆍ운영 조례 일부개정조례안｣, 박상돈 의원님이 대표 발의하신 ｢청주시 전통시장 및 상점가 육성과 관리에 관한 조례 일부개정조례안｣, 이유자 의원님이 대표 발의하신 ｢청주시 재능기부 활성화 조례안｣, 김성택 의원님이 대표 발의하신 ｢청주시 문화산업진흥재단 설립 및 운영ㆍ지원 조례 일부개정조례안｣, 박노학 의원님이 대표 발의하신 ｢청주시 용역과제심의위원회 설치 및 운영 조례 일부개정조례안｣, 임기중 의원님이 대표 발의하신 ｢청주시 양성평등 기본 조례 일부개정조례안｣, 홍순평 의원님이 대표 발의하신 ｢청주시 재난취약계층 지원 조례안｣, 김병국 의원님이 대표 발의하신 ｢청주시의회 행정사무감사 및 조사에 관한 조례 일부개정조례안｣과 ｢청주시의회 회기운영 [MASK] 등에 관한 조례 일부개정조례안｣, 박현순 의원님이 대표 발의하신 ｢청주시 안전관리민관협력위원회 구성 및 운영 조례안｣, ｢청주시 빈집 정비 조례안｣, 김태수 의원님이 대표 발의하신 ｢청주시 공공디자인 진흥 조례안｣, 신언식 의원님이 대표 발의하신 ｢청주시 청소년 노동인권 보호 및 증진 조례안｣, ｢청주시 한부모가족 지원 조례안｣, 박금순 의원님이 대표 발의하신 ｢청주시 경력단절여성등의 경제활동 촉진에 관한 조례안｣, 안성현 의원님이 대표

In [20]:
context = 0
sentence = 11

print('원본 데이터 :')
print(train['context'][context].split('.')[sentence])

print('\n-----------------------------------------\n')

print('증식된 데이터 :')
print(dict_random_insertion['context'][context].split('.')[sentence])

원본 데이터 :
  다음은 완주군수로부터 제출된 안건입니다

-----------------------------------------

증식된 데이터 :
 다음은 완주군수로부터 공식 제출된 안건입니다


# Random Replacement

In [ ]:
dict_random_replacement = {
    'id':list(train['id']),
    'title':list(train['title']),
    'region':list(train['agenda']),
    'context':[],
    'summary':list(train['summary'])
}

unmasker = pipeline('fill-mask', model='klue/roberta-base')

for input_text in tqdm(list(train['context'])):

  new_text = ''
  
  sentence_list = sent_tokenize(input_text)

  for sentence in sentence_list:

    orig_text_list = sentence.split()
    len_input = len(orig_text_list)
    

    if len_input == 1:
      new_text += ' ' + sentence

    elif len_input <= 3:
      orig_word = orig_text_list[1]
      new_text_list = orig_text_list.copy()
      new_text_list[1] = '[MASK]'
      new_mask_sent = ' '.join(new_text_list)

      try:
        augmented_text_list = unmasker(new_mask_sent)
        for res in augmented_text_list:
          if res['token_str'] != orig_word:
            augmented_text = res['sequence']
            break

            new_augmented_text = ' ' + augmented_text
            new_text += new_augmented_text
            new_text = new_text.lstrip(' ')

      except:
        print(f'오류 : {new_mask_sent}')
        new_text += ' ' + sentence
        pass

    else:
      rand_idx = random.randint(1, len_input-2)

      orig_word = orig_text_list[rand_idx]
      new_text_list = orig_text_list.copy()
      new_text_list[rand_idx] = '[MASK]'
      new_mask_sent = ' '.join(new_text_list)

      try:
        augmented_text_list = unmasker(new_mask_sent)
        for res in augmented_text_list:
          if res['token_str'] != orig_word:
            augmented_text = res['sequence']
            break
      except:
        print(f'오류 : {new_mask_sent}')
        new_text += ' ' + sentence
        pass

      new_augmented_text = ' ' + augmented_text
      new_text += new_augmented_text
      new_text = new_text.lstrip(' ')

  dict_random_replacement['context'].append(new_text+'.')

df_random_replacement = pd.DataFrame(dict_random_replacement)
df_random_replacement.to_csv('df_random_replacement.csv')
df_random_replacement.head()

  0%|          | 0/2994 [00:00<?, ?it/s]

비학습 : 유의식 의원님이 발의한 완주군 자치분권 촉진 및 지원에 관한 조례, 소완섭 의원님이 발의한 완주문화원 육성 및 지원에 관한 조례, 김재천 의원님이 발의한 [MASK] 민주평화통일자문회의 완주군협의회 지원에 관한 조례가 제출되었고, 전체 의원님이 공동 발의한 완주군의정회 지원 조례 폐지 조례안, 완주군의회 의원 윤리강령 및 행동강령 조례 일부개정 조례안, 완주군의회 의원 의정활동비 등 지급에 관한 조례 일부개정 조례안이 제출되어 해당 상임위원회로 회부하였습니다.
비학습 : 의원 [MASK] 이의 있으십니까?(『없습니다.


Token indices sequence length is longer than the specified maximum sequence length for this model (676 > 512). Running this sequence through the model will result in indexing errors


비학습 : 다음은 의원 [MASK] 의안으로 이재길 의원님이 대표 발의하신 청주시장 및 관계공무원 출석요구의 건, 안흥수 의원님이 대표 발의하신 ｢청주시 의사상자 등 예우 및 지원에 관한 조례안｣, 변창수 의원님이 대표 발의하신 ｢청주시 장애인공무원 편의지원 조례안｣, 임기중 의원님이 대표 발의하신 ｢청주시 출산장려 및 양육에 대한 지원 조례 일부개정조례안｣, 서지한 의원님이 대표 발의하신 ｢청주시립요양병원 설치 및 운영에 관한 조례 일부개정조례안｣, 육미선 의원님이 대표 발의하신 ｢청주시 마을방송시스템 설치 지원에 관한 조례안｣, 최진현 의원님이 대표 발의하신 ｢청주시 주민자치센터 설치 및 운영 조례 일부개정조례안｣, 맹순자 의원님이 대표 발의하신 ｢청주시 지방재정계획심의위원회 설치ㆍ운영 조례 일부개정조례안｣, 박상돈 의원님이 대표 발의하신 ｢청주시 전통시장 및 상점가 육성과 관리에 관한 조례 일부개정조례안｣, 이유자 의원님이 대표 발의하신 ｢청주시 재능기부 활성화 조례안｣, 김성택 의원님이 대표 발의하신 ｢청주시 문화산업진흥재단 설립 및 운영ㆍ지원 조례 일부개정조례안｣, 박노학 의원님이 대표 발의하신 ｢청주시 용역과제심의위원회 설치 및 운영 조례 일부개정조례안｣, 임기중 의원님이 대표 발의하신 ｢청주시 양성평등 기본 조례 일부개정조례안｣, 홍순평 의원님이 대표 발의하신 ｢청주시 재난취약계층 지원 조례안｣, 김병국 의원님이 대표 발의하신 ｢청주시의회 행정사무감사 및 조사에 관한 조례 일부개정조례안｣과 ｢청주시의회 회기운영 등에 관한 조례 일부개정조례안｣, 박현순 의원님이 대표 발의하신 ｢청주시 안전관리민관협력위원회 구성 및 운영 조례안｣, ｢청주시 빈집 정비 조례안｣, 김태수 의원님이 대표 발의하신 ｢청주시 공공디자인 진흥 조례안｣, 신언식 의원님이 대표 발의하신 ｢청주시 청소년 노동인권 보호 및 증진 조례안｣, ｢청주시 한부모가족 지원 조례안｣, 박금순 의원님이 대표 발의하신 ｢청주시 경력단절여성등의 경제활동 촉진에 관한 조례안｣, 안성현 의원님이 대표 발

In [72]:
context = 3
sentence = 4

print('원본 데이터 :')
print(train['context'][context].split('.')[sentence])

print('\n-----------------------------------------\n')

print('증식된 데이터 :')
print(dict_random_replacement['context'][context].split('.')[sentence])

원본 데이터 :
  다음 제2차 본회의는 9월 4일 오전 10시에 개의하겠습니다

-----------------------------------------

증식된 데이터 :
 다음 제2차 본회의는 내일 4일 오전 10시에 개의하겠습니다


In [66]:
train['context'][context]

'다음은 의사일정 제2항 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상정합니다.  회의록 서명의원으로는 최등원 의원님과 박웅배 의원님을 선출하고자 하는데 의원여러분 이의 있습니까?(『없습니다』하는 의원 있음)이의가 없으므로 가결되었음을 선포합니다.  그럼 두 분 의원님께서는 본 회기동안 수고하여 주시기 바랍니다'

In [67]:
dict_random_replacement['context'][context]

'다음은 의사일정 중 제207회 완주군의회 임시회 회의록 서명의원 선출의 건을 상정합니다. 회의록 서명의원으로는 최등원 의원님과 박웅배 의원님을 추천 하는데 의원여러분 이의 있습니까? ( 『 없습니다 』 하는 의원 있음 ) 이의가 없으므로 이를 선포합니다. 그럼 두 분 의원님께서는 본 회기동안 지켜봐 주시기 바랍니다'

# RI + SR

In [ ]:
dict_RI_SR = {
    'id':list(train['id']),
    'title':list(train['title']),
    'region':list(train['agenda']),
    'context':[],
    'summary':list(train['summary'])
}

unmasker = pipeline('fill-mask', model='klue/roberta-base')

for input_text in tqdm(list(train['context'])):

  new_text = ''
  
  sentence_list = sent_tokenize(input_text)

  for sentence in sentence_list:

    orig_text_list = sentence.split()
    len_input = len(orig_text_list)

    if len_input == 1:
      new_text += ' ' + sentence      

    elif len_input <= 2:

      orig_word = orig_text_list[1]
      new_text_list = orig_text_list.copy()
      new_text_list[1] = '[MASK]'

      new_text_list = orig_text_list[:1] + ['[MASK]'] + orig_text_list[1:]
      
      new_mask_sent = ' '.join(new_text_list)

      try:
        augmented_text_list = unmasker(new_mask_sent)
        for res in augmented_text_list:
          if res['token_str'] != orig_word:
            augmented_text = res['sequence']
            break

        new_augmented_text = ' ' + augmented_text
        new_text += new_augmented_text
        new_text = new_text.lstrip(' ')

      except:
        print(f'오류 : {new_mask_sent}')
        new_text += ' ' + sentence
        pass

    else:
      rand_idx = random.randint(1, len_input-2)

      orig_word = orig_text_list[rand_idx]
      new_text_list = orig_text_list.copy()
      new_text_list[rand_idx] = '[MASK]'

      new_text_list = orig_text_list[:rand_idx] + ['[MASK]'] + orig_text_list[rand_idx:]
      
      new_mask_sent = ' '.join(new_text_list)

      try:
        augmented_text_list = unmasker(new_mask_sent)
        for res in augmented_text_list:
          if res['token_str'] != orig_word:
            augmented_text = res['sequence']
            break

        new_augmented_text = ' ' + augmented_text
        new_text += new_augmented_text
        new_text = new_text.lstrip(' ')

      except:
        print(f'오류 : {new_mask_sent}')
        new_text += ' ' + sentence
        pass

  dict_RI_SR['context'].append(new_text+'.')

df_RI_SR = pd.DataFrame(dict_RI_SR)
df_RI_SR.to_csv('df_RI_SR.csv')
df_RI_SR.head()